In [ ]:
pip install pyvirtualdisplay

In [ ]:
pip install pygame

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

import pickle as pickle
from tqdm.notebook import tqdm



In [ ]:
env = gym.make('FrozenLake-v1', map_name="4x4", is_slippery=False, render_mode="rgb_array")

creating custom grid like this

In [ ]:
desc = ["SFFF","FHFH","FFHH","HFFG"]

In [ ]:
env = gym.make('FrozenLake-v1', desc=desc, is_slippery=False)

In [ ]:
print(env.observation_space)

Discrete(16)


In [ ]:
print(env.observation_space.sample())

7


In [ ]:
print(env.action_space)

Discrete(4)


In [ ]:
print(env.action_space.sample())

2


In [ ]:
print(env.action_space.n)

4


In [ ]:
state_space = env.observation_space.n
print(state_space)

16


In [ ]:
action_space = env.action_space.n
print(action_space)

4


In [ ]:
def Q_table(state_space, action_space):
  qtable = np.zeros((state_space, action_space))
  return qtable

In [ ]:
qtable = Q_table(state_space, action_space)

In [ ]:
qtable = Q_table(state_space, action_space)
display(qtable)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
def greedy_pol(qtable, state):
  action = np.argmax(qtable[state][:])
  return action

In [ ]:
def epsilon_greedy_pol(qtable, state, epsilon):
  random = np.random.uniform(0,1)
  if random > epsilon:
    action = greedy_pol(qtable, state)
  else:
    action = env.action_space.sample()
  return action

Hyperparameters

In [ ]:
n_training_eps = 10000
lr = 0.7
n_eval_eps = 100
env_id = "FrozenLake-v1"
max_steps = 99
gamma = 0.05
eval_seeds = []
max_epsilon = 1.0
min_epsilon = 0.005
decay_rate = 0.0005

ϵ ep =ϵ min +(ϵ max − ϵ min)⋅e
−decay_rate⋅ep


In [ ]:
def train(n_training_eps, min_epsilon, max_epsilon, decay_rate, env, max_steps, qtable):
  for ep in tqdm(range(n_training_eps)):
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*ep)
    state,info = env.reset()

    step=0
    terminated = False
    truncated = False

    for step in range(max_steps):
      action = epsilon_greedy_pol(qtable,state,epsilon)

      new_state, reward, terminated, truncated, info= env.step(action)
      qtable[state][action] = qtable[state][action] +lr*(reward + gamma*np.max(qtable[new_state]) - qtable[state][action])

      if terminated or truncated:
        break

      state = new_state
  return qtable

In [ ]:
q_table_frozen = train(n_training_eps, min_epsilon, max_epsilon, decay_rate, env, max_steps, qtable)


  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
def eval(env, max_steps, n_eval_eps, seed, Q):
  ep_rewards = []
  for ep in tqdm(range(n_eval_eps)):
    if seed:
      state, infp = env.reset(seed=seed[ep])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      action = greedy_pol(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    ep_rewards.append(total_rewards_ep)
  mean_reward = np.mean(ep_rewards)
  std_reward = np.std(ep_rewards)

  return mean_reward, std_reward

In [ ]:
mean_reward, std_reward = eval(env, max_steps, n_eval_eps, eval_seeds, q_table_frozen)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

  0%|          | 0/100 [00:00<?, ?it/s]

Mean_reward=1.00 +/- 0.00
